Packages

In [1]:
!pip install google-generativeai streamlit pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.5 MB/s eta 0:00:00


GEMINI_API_KEY

In [2]:
import yaml

with open('api_keys.yml', 'r') as file:
    api_creds = yaml.safe_load(file)

In [3]:
api_creds.keys()

dict_keys(['OPENAI_API_KEY', 'NGORK_AUTH_TOKEN', 'GEMINI_API_KEY', 'TAVILY_API_KEY', 'GROQ_API_KEY', 'OPENBB_PAT', 'FMP_API_KEY', 'WANDB_API_KEY', 'Hugginbface_API_KEY'])

Creating app.py file for running with Streamlit

In [29]:
%%writefile app.py
import os
import io
from typing import Optional
import streamlit as st
from PIL import Image
import google.generativeai as genai

st.set_page_config(page_title="Visual QA Using Streamlit", page_icon="🖼️", layout="centered")
st.title("🖼️ Visual Q&A (Gemini)")

st.markdown(
    "Upload an image, enter a question and let a multimodal model give answer.")

# API key handling
default_key = os.getenv("GOOGLE_API_KEY", "")
api_key = st.text_input("Google API Key", value=default_key, type="password", help="Stored in env as GOOGLE_API_KEY")
if not api_key:
    st.info("Enter your Google API key to continue.", icon="🔑")
    st.stop()

genai.configure(api_key=api_key)

model_name = st.selectbox(
    "Model",
    ["gemini-1.5-flash", "gemini-1.5-pro"],
    index=0,
    help="Flash is fast and cost-efficient; Pro is more accurate but expensive."
)

uploaded_file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])
image = None
if uploaded_file is not None:
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption=f"Uploaded Image - {getattr(uploaded_file, 'name', 'image')}", use_column_width=True)

question = st.text_area("Your question about the image", placeholder="e.g., What is written on the sign?")
go = st.button("Ask")
if go:
    if uploaded_file is None:
        st.error("Please upload an image.", icon="⚠️")
        st.stop()
    if not question.strip():
        st.error("Please enter a question.", icon="⚠️")
        st.stop()
    # load image in memory
    image = Image.open(uploaded_file).convert("RGB")
    image_bytes = io.BytesIO()
    image.save(image_bytes, format="JPEG")
    image_bytes = image_bytes.getvalue()

    #Gemini multimodel prompt: image + text
    model = genai.GenerativeModel(model_name)

    with st.spinner("Generating answer..."):
        try:
          resp = model.generate_content(
              [
                  {"mime_type":"image/png", "data":image_bytes},
                  question.strip()
              ]
          )
          answer = (resp.text or "(No text response)").strip()
        except Exception as e:
          answer = f"Error from model: {e}"
    st.subheader("Answer")
    st.write(answer)

    st.caption("Tip: Try questions about text in the image, objects, scenes, chats, etc.")


Overwriting app.py


In [30]:
!streamlit run app.py --server.port=8989 &>./logs.txt &

In [31]:
from pyngrok import ngrok
import yaml

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken
# Get your authtoken from `ngrok_credentials.yml` file
# with open('ngrok_credentials.yml', 'r') as file:
#     NGROK_AUTH_TOKEN = yaml.safe_load(file)
ngrok.set_auth_token(api_creds['NGORK_AUTH_TOKEN'])

# Open an HTTPs tunnel on port XXXX which you get from your `logs.txt` file
ngrok_tunnel = ngrok.connect(8989)
print("Streamlit App:", ngrok_tunnel.public_url)

Streamlit App: https://fba18d56d935.ngrok-free.app


In [32]:
ngrok.kill()

In [33]:
!ps -ef | grep streamlit

root        3061       1  6 22:53 ?        00:00:04 /usr/bin/python3 /usr/local/bin/streamlit run app.py --server.port=8989
root        3352     185  0 22:54 ?        00:00:00 /bin/bash -c ps -ef | grep streamlit
root        3354    3352  0 22:54 ?        00:00:00 grep streamlit


In [34]:
!sudo kill -9 3061